In [18]:
import os
import shutil

image_folder = '/home/quangsang/test/nighttime_normalized'
new_folder = 'cam8'

os.makedirs(os.path.join(new_folder, 'images'), exist_ok=True)
os.makedirs(os.path.join(new_folder, 'labels'), exist_ok=True)

def copy_files(file_list, src_folder, dest_folder):
    for image_file in file_list:
        image_path = os.path.join(src_folder, image_file)
        label_path = os.path.join(src_folder, image_file.replace('.jpg', '.txt'))

        shutil.copy(image_path, os.path.join(dest_folder, 'images', image_file))
        if os.path.exists(label_path):
            shutil.copy2(label_path, os.path.join(dest_folder, 'labels', image_file.replace('.jpg', '.txt')))

image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg') and f.startswith('cam_08_')]

copy_files(image_files, image_folder, new_folder)

In [26]:
import os
from PIL import Image

images_dir = 'cam10/images'
labels_dir = 'cam10/labels'
output_dir = 'cropped_cam10/images'
output_labels_dir = 'cropped_cam10/labels'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if not os.path.exists(output_labels_dir):
    os.makedirs(output_labels_dir)

for image_name in os.listdir(images_dir):
    if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(images_dir, image_name)
        label_path = os.path.join(labels_dir, os.path.splitext(image_name)[0] + '.txt')
        
        if not os.path.isfile(label_path):
            continue
        
        with Image.open(image_path) as img:
            width, height = img.size
            with open(label_path, 'r') as file:
                for idx, line in enumerate(file):
                    parts = line.strip().split()
                    if len(parts) != 5:
                        continue
                    class_id, x_center, y_center, bbox_width, bbox_height = map(float, parts)
                    
                    x_center *= width
                    y_center *= height
                    bbox_width *= width
                    bbox_height *= height
                    
                    x1 = int(x_center - bbox_width / 2)
                    y1 = int(y_center - bbox_height / 2)
                    x2 = int(x_center + bbox_width / 2)
                    y2 = int(y_center + bbox_height / 2)
                    
                    cropped_img = img.crop((x1, y1, x2, y2))
                    output_image_name = f"{os.path.splitext(image_name)[0]}_crop_{idx}_class_{int(class_id)}.jpg"
                    cropped_img.save(os.path.join(output_dir, output_image_name))
                    
                    # Xuất tệp nhãn cho ảnh đã cắt
                    output_label_name = f"{os.path.splitext(image_name)[0]}_crop_{idx}_class_{int(class_id)}.txt"
                    with open(os.path.join(output_labels_dir, output_label_name), 'w') as label_file:
                        label_file.write(f"{int(class_id)} {x_center} {y_center} {bbox_width} {bbox_height}\n")